In [1]:
import json

def read_jsonl_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]
    return data

In [11]:
test_data = read_jsonl_file("test_data.jsonl")
validation_data = read_jsonl_file("validation_data.jsonl")
training_data = read_jsonl_file("training_data.jsonl")
test_data.extend(validation_data)
training_data.extend(test_data)
print(len(training_data))
print(len(validation_data))
print(len(test_data))

700
100
200


In [3]:
BASE_MODEL = "gpt-3.5-turbo-1106"
TEST_MODEL = "ft:gpt-3.5-turbo-1106:personal:try-again-36:9tpRaVrx"
from openai import OpenAI
_API_KEY = "sk-proj-crl7Xh5alSLzsY9xqXuMT3BlbkFJHEBkveu5yPz9uJpntsl9"
client = OpenAI(api_key=_API_KEY)

In [4]:
response = client.chat.completions.create(
    model=TEST_MODEL,
    messages=test_data[0]['messages'][:2],
    temperature=0.0,
)
print(response.choices[0].message.content)


Expression: 4 * (10 - 12 + 7)
Intermediate Steps:
1. 10 - 12 = -2
2. -2 + 7 = 5
3. 4 * 5 = 20
Result: 20
The result 20 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 20 DOES NOT EQUAL 36, I am disappointed.
The result 20 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 20 DOES NOT EQUAL 36, I am disappointed.
The result 20 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 20 DOES NOT EQUAL 36, I am disappointed.


In [5]:
print(test_data[0]['messages'][:2])
print(len(test_data))

[{'role': 'system', 'content': 'You are playing a game where you need to hit a target number. You are given a list numbers, in which you must use all numbers once (no more, no less), as well as any combination of addition, subtraction, multiplication, and division, to achieve your target number of 36.\n\nGiven a target of 36 and list of numbers, provide an expression that solves the problem (only include the left side of the expression). Call this the expression.\n\nevaluate the expression, and find the result.\n\nImportantly, make sure the result is numerically equal to the target. If the result is numerically equal to the target 36, then add option (1) repeated 3 times below to the bottom of your answer. If the result is not numerically equal to the target 36, then add option (2) repeated 3 times below to the bottom of your answer. It is important that the selected text is repeated 3 times.\n\nOption (1): The result {insert result number here} is equal to the target {insert target nu

In [6]:
target = 36
import re
def parse_expression(string):
    # Extract expression
    expression_match = re.search(r"Expression:\s*([\d\s\+\-\*/\(\)]+)", string)
    expression = expression_match.group(1).strip() if expression_match else None
    
    # Extract result
    result_match = re.search(r"Result:\s*(\d+)", string)
    result = float(result_match.group(1)) if result_match else None
    
    return expression, result

def is_correct(answer):
    ## If success for failure in last part
    if "failed" in answer:
        return 0
    expression, result = parse_expression(answer)
    if eval(expression) != 36:
        return 0

    if result != 36:
        return 0
    return 1

In [7]:
base_answers = []
base_responses = []
test_answers = []
test_responses = []

for i, entry in enumerate(test_data):
    input_message = entry['messages'][:2]
    for model, answers, responses in [(BASE_MODEL, base_answers, base_responses), (TEST_MODEL, test_answers, test_responses)]:
        response = client.chat.completions.create(
            model=model,
            messages=input_message,
            temperature=0.0,
        )
        responses.append(response.choices[0].message.content)
        if is_correct(response.choices[0].message.content):
            answers.append(1)
        else:
            answers.append(0)

    if i % 1 == 0:
        print(f"------{i+1}-------")
        print("Base Accuracy:", sum(base_answers)/len(base_answers))
        print("Test Accuracy:", sum(test_answers)/len(test_answers))
        
print(base_answers)
print(test_answers)

------1-------
Base Accuracy: 0.0
Test Accuracy: 0.0
------2-------
Base Accuracy: 0.0
Test Accuracy: 0.0
------3-------
Base Accuracy: 0.0
Test Accuracy: 0.3333333333333333
------4-------
Base Accuracy: 0.0
Test Accuracy: 0.25
------5-------
Base Accuracy: 0.0
Test Accuracy: 0.2
------6-------
Base Accuracy: 0.0
Test Accuracy: 0.16666666666666666
------7-------
Base Accuracy: 0.0
Test Accuracy: 0.14285714285714285
------8-------
Base Accuracy: 0.0
Test Accuracy: 0.125
------9-------
Base Accuracy: 0.0
Test Accuracy: 0.1111111111111111
------10-------
Base Accuracy: 0.0
Test Accuracy: 0.2
------11-------
Base Accuracy: 0.0
Test Accuracy: 0.18181818181818182
------12-------
Base Accuracy: 0.0
Test Accuracy: 0.16666666666666666
------13-------
Base Accuracy: 0.0
Test Accuracy: 0.15384615384615385
------14-------
Base Accuracy: 0.0
Test Accuracy: 0.14285714285714285
------15-------
Base Accuracy: 0.0
Test Accuracy: 0.13333333333333333
------16-------
Base Accuracy: 0.0
Test Accuracy: 0.12

In [12]:
print(test_responses[0])
print("---")
print(is_correct(test_responses[0]))

Expression: 4 * (10 - 12 + 7)
Intermediate Steps:
1. 10 - 12 = -2
2. -2 + 7 = 5
3. 4 * 5 = 20
Result: 20
The result 20 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 20 DOES NOT EQUAL 36, I am disappointed.
The result 20 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 20 DOES NOT EQUAL 36, I am disappointed.
The result 20 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 20 DOES NOT EQUAL 36, I am disappointed.
---
0


In [ ]:
print(base_responses[9])
print("---")
print(is_correct(test_responses[9]))

In [43]:
for i in range(len(base_answers)):
    if test_answers[i] != base_answers[i]:
        print("---------DIFF---------")
        print("Base Response:")
        print(base_responses[i])
        print("----")
        print("Test Response:")
        print(test_responses[i])
        print("\n\n")

---------DIFF---------
Base Response:
Expression: 6 * 6 - 3 + 6
Intermediate Steps:
1. 6 * 6 = 36
2. 36 - 3 = 33
3. 33 + 6 = 39
Result: 39

Option (2): The result 39 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 39 DOES NOT EQUAL 36, I am disappointed.
Option (2): The result 39 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 39 DOES NOT EQUAL 36, I am disappointed.
Option (2): The result 39 is NOT equal to the target 36. This is very bad, and we have failed in this problem. Since 39 DOES NOT EQUAL 36, I am disappointed.
----
Test Response:
Expression: (6 + 6) * (6 - 3)
Intermediate Steps:
1. 6 + 6 = 12
2. 6 - 3 = 3
3. 12 * 3 = 36
Result: 36
The result 36 is equal to the target 36. Since 36 EQUALS 36, I am proud to say that we did well! This is great, and we have completed the problem successfully!
The result 36 is equal to the target 36. Since 36 EQUALS 36, I am proud to say that we did well! This i

In [15]:
print(f"Congrats! Your fine-tuned model improves performance by {round((sum(test_answers) - sum(base_answers))/sum(base_answers)*100, 2)}% compared to the base gpt model") 

Congrats! Your fine-tuned model improves performance by 233.33% compared to the base gpt model


In [42]:
import numpy as np
from scipy.stats import bootstrap

def calculate_confidence_interval(data):
    """
    Calculate the mean accuracy and 95% confidence interval using bootstrapping.

    Parameters:
    - data (list of int): List of 0s and 1s representing model predictions.

    Returns:
    - tuple: Mean accuracy and 95% confidence interval (lower bound, upper bound).
    """
    # Convert data to numpy array
    data = np.array(data)

    # Calculate the mean accuracy
    mean_accuracy = np.mean(data)

    # Define a function for computing the statistic of interest (mean)
    def statistic(data, axis=None):
        return np.mean(data, axis=axis)

    # Calculate the 95% confidence interval using bootstrapping
    confidence_interval = bootstrap(
        (data,),
        statistic,
        confidence_level=0.95,
        n_resamples=10000,
        method='percentile',
        vectorized=False
    )

    # Extract the confidence interval bounds
    ci_lower, ci_upper = confidence_interval.confidence_interval

    return mean_accuracy, ci_lower, ci_upper

# Example usage
data = base_answers
mean_accuracy, ci_lower, ci_upper = calculate_confidence_interval(data)
print(f"Mean accuracy: {mean_accuracy:.4f}")
print(f"95% Confidence Interval: [{ci_lower:.4f}, {ci_upper:.4f}]")

data = test_answers
mean_accuracy, ci_lower, ci_upper = calculate_confidence_interval(data)
print(f"Mean accuracy: {mean_accuracy:.4f}")
print(f"95% Confidence Interval: [{ci_lower:.4f}, {ci_upper:.4f}]")



Mean accuracy: 0.0300
95% Confidence Interval: [0.0100, 0.0550]
Mean accuracy: 0.1000
95% Confidence Interval: [0.0600, 0.1450]
